## Importo le librerie

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

### Importo la pagina

Importo l'url per terreni in veneto

In [2]:
regioni = ['abruzzo', 'basilicata', 'campania', 'calabria', 'emilia-romagna', 'friuli-venezia-giulia', 'lazio','liguria', 'lombardia','marche','molise','piemonte','puglia','sardegna','sicilia','toscana', 'trentino-alto-adige','umbria','valle-d-aosta','veneto']
tipo = 'case'
#regione = regioni[-1]

#url = f"https://www.immobiliare.it/vendita-{tipo}/{regione}/?criterio=rilevanza&pag=1"
#page = requests.get(url)
#soup = BeautifulSoup(page.content, "html.parser")

In [3]:
## Controllo il massimo numero di pagine per la regione scelta
#num_pages = soup.find_all('div', class_="in-pagination__item is-mobileHidden in-pagination__item--disabled")
#max_num_pages = int(num_pages[2].text)
#max_num_pages

In [4]:
#root = f"https://www.immobiliare.it/vendita-{tipo}/{regione}/?criterio=rilevanza&pag="
price_list = []
adv_list = []
region = []
locali = []
superficie = []
bagni = []

In [5]:
for reg in regioni:
    
    url = f"https://www.immobiliare.it/vendita-{tipo}/{reg}/?criterio=rilevanza&pag=1"
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")

    num_pages = soup.find_all('div', class_="in-pagination__item is-mobileHidden in-pagination__item--disabled")
    max_num_pages = int(num_pages[2].text)
    max_num_pages

    root = f"https://www.immobiliare.it/vendita-{tipo}/{reg}/?criterio=rilevanza&pag="
        
    
    for i in range(1,max_num_pages+1):
        
        url = f"{root}{i}"
        print(f"Sto esaminando:\t{url}")
        page = requests.get(url)
        soup = BeautifulSoup(page.content, "html.parser")
        house_card = soup.find_all('div', class_="nd-mediaObject__content in-card__content in-realEstateListCard__content")
        for _ in house_card:
            price_list.append(_.find('li').text)
            adv_list.append(_.find('a', class_="in-card__title").attrs.get('title'))
            region.append(reg)
            if _.find('li', {"class" : "nd-list__item in-feat__item", "aria-label": "locali"}) is not None:
                locali.append(_.find('li', {"class" : "nd-list__item in-feat__item", "aria-label": "locali"}).text)
            else:
                locali.append(np.nan)
        
            if _.find('li', {"class" : "nd-list__item in-feat__item", "aria-label": "superficie"}) is not None:
                superficie.append(_.find('li', {"class" : "nd-list__item in-feat__item", "aria-label": "superficie"}).text)
            else:
                superficie.append(np.nan)
        
            if _.find('li', {"class" : "nd-list__item in-feat__item", "aria-label": "bagni"}) is not None:
                bagni.append(_.find('li', {"class" : "nd-list__item in-feat__item", "aria-label": "bagni"}).text)
            else:
                bagni.append(1)

Sto esaminando:	https://www.immobiliare.it/vendita-case/abruzzo/?criterio=rilevanza&pag=1
Sto esaminando:	https://www.immobiliare.it/vendita-case/abruzzo/?criterio=rilevanza&pag=2
Sto esaminando:	https://www.immobiliare.it/vendita-case/abruzzo/?criterio=rilevanza&pag=3
Sto esaminando:	https://www.immobiliare.it/vendita-case/abruzzo/?criterio=rilevanza&pag=4
Sto esaminando:	https://www.immobiliare.it/vendita-case/abruzzo/?criterio=rilevanza&pag=5
Sto esaminando:	https://www.immobiliare.it/vendita-case/abruzzo/?criterio=rilevanza&pag=6
Sto esaminando:	https://www.immobiliare.it/vendita-case/abruzzo/?criterio=rilevanza&pag=7
Sto esaminando:	https://www.immobiliare.it/vendita-case/abruzzo/?criterio=rilevanza&pag=8
Sto esaminando:	https://www.immobiliare.it/vendita-case/abruzzo/?criterio=rilevanza&pag=9
Sto esaminando:	https://www.immobiliare.it/vendita-case/abruzzo/?criterio=rilevanza&pag=10
Sto esaminando:	https://www.immobiliare.it/vendita-case/abruzzo/?criterio=rilevanza&pag=11
Sto esam

In [6]:
df = pd.DataFrame(
    {'advertisement': adv_list,
     'region': region,
     'rooms': locali,
     'toilets': bagni,
     'area': superficie,
     'price': price_list
    })

df.sample(10)

,advertisement,region,rooms,toilets,area,price
31305,"Trilocale via Camillo Benso di Cavour 85, San ...",toscana,3,2,77m²,€ 415.000
31896,"Appartamento buono stato, Ospedaletto, Pisa",toscana,5+,1,200m²,€ 450.000
28487,"Appartamento corso Giuseppe Garibaldi 107, Cen...",sicilia,5+,3+,210m²,Prezzo su richiesta
16470,"Quadrilocale via Angela Poeta 7, Conca Fiorita...",lombardia,4,2,145m²,€ 440.000
33116,"Quadrilocale via della Canova, Gardolo, Trento",trentino-alto-adige,4,2,92m²,€ 293.000
10870,"Villa unifamiliare Strada di Portopiccolo, Dui...",friuli-venezia-giulia,5+,3+,592m²,€ 5.200.000
27341,"Quadrilocale via Gavino Alivia 14, Luna e Sole...",sardegna,4,2,120m²,€ 147.000
2138,"Appartamento all'asta 85055 Picerno Italia, Pi...",basilicata,5,1,106m²,"da € 54.525,00"
4232,"Bilocale via San Domenico 29, Vomero, Napoli",campania,2,1,70m²,€ 310.000
25708,"Bilocale Lungomare 9 Maggio 80, San Girolamo, ...",puglia,2,1,54m²,€ 220.000


In [7]:
df['city'] = df["advertisement"].apply(lambda x: x.split(',')[-1])


In [10]:
df = df[['region','city','area','rooms','toilets','price']]
df.sample(10)

,region,city,area,rooms,toilets,price
5727,campania,Capaccio Paestum,40m²,2,1,€ 59.000
23079,piemonte,Novara,78m²,3,1,€ 85.000
35513,umbria,Baschi,500m²,5+,3+,€ 620.000
12944,lazio,Roma,150m²,5,2,€ 465.000
30853,toscana,Lucca,75m²,4,1,€ 149.000
17226,lombardia,Milano,35m²,NaN,1,€ 270.000
30539,toscana,Firenze,87m²,4,1,€ 320.000
26334,sardegna,Assemini,90m²,5,2,€ 195.000
10800,friuli-venezia-giulia,San Daniele del Friuli,81m²,3,1,€ 104.000
628,abruzzo,Alanno,120m²,4,1,€ 30.000


In [11]:
df.to_csv('house_prices_italy.csv')